# Tensorflow Extended Test
---

There are two main goals with this notebook: 

 1. Establish that TFHub works within the context of a TFX training pipeline 
 2. Use the given query with the BQ Example Gen component and then feed this data through the pipeline. 

In [1]:
import os
import pprint
import tempfile
import urllib

import pandas as pd
from sklearn.model_selection import train_test_split

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components.example_gen.custom_executors import parquet_executor 
from tfx.components.example_gen.component import FileBasedExampleGen
from tfx.extensions.google_cloud_big_query.example_gen.component import BigQueryExampleGen

from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto import example_gen_pb2

from tensorflow_metadata.proto.v0 import schema_pb2 
from tensorflow_metadata.proto.v0 import statistics_pb2
from tensorflow_metadata.proto.v0 import anomalies_pb2

from ml_metadata import metadata_store
from ml_metadata.proto import metadata_store_pb2

import tensorflow_data_validation as tfdv 
import tensorflow_transform as tft
import tensorflow_model_analysis as tfma

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input

Let's use a small amount of data just to speed things up a bit. 

In [2]:
_pipeline_root = tempfile.mkdtemp(prefix='pipeline')
_pipeline_name = 'interactive_pipeline'

context = InteractiveContext(pipeline_name=_pipeline_name, pipeline_root=_pipeline_root)

In [3]:
connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = '/tmp/pipelineuh946s31/metadata.sqlite' # REPLACE with new location
connection_config.sqlite.connection_mode = 3 #READWRITE_OPENCREATE
store = metadata_store.MetadataStore(connection_config)
base_dir = connection_config.sqlite.filename_uri.split('metadata.sqlite')[0]

**NOTE:** Not really sure what I should set hash_buckets as when there's only one split? Not sure it matters

In [4]:
output = example_gen_pb2.Output(
             split_config=example_gen_pb2.SplitConfig(splits=[
                 example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=10)
             ],
    ))

**NOTE:** This is the completely unaltered query

In [5]:
query = """
SELECT 
  synopsis,
  tags
FROM `ml-sandbox-101.metadata_sky.merlin_movie_series_data_small`
LIMIT 1000
"""

**NOTE:** Need to figure out if this is how we do this when running with beam/kubeflow. Need to establish 
naming conventions for temp locations

In [6]:
# Beam args to use BigQueryExampleGen with Beam DirectRunner.
BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS = [
   '--project=' + 'ml-sandbox-101',
   '--temp_location=' + os.path.join('gs://', 'metadata-bucket-sky', 'tmp'),
   ]

In [7]:
example_gen = BigQueryExampleGen(query=query, output_config=output)
context.run(example_gen, beam_pipeline_args=['--project', 'ml-sandbox-101', '--temp_location', 'gs://metadata-bucket-sky/tmp'])

/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1881: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(


ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/tmp/pipelineuss5158a/BigQueryExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "BigQueryExampleGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

In [10]:
def make_dataset(examples, split):
    """
    Create a dataset from examples artifact
    """
    uri = os.path.join(examples.get()[0].uri, split)
    
    tfrecord_filenames = [os.path.join(uri, name) for name in os.listdir(uri)]
    dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
    
    return dataset

In [11]:
train_dataset = make_dataset(example_gen.outputs['examples'], 'train')

print('TRAIN DATASET SIZE: {}'.format(train_dataset.reduce(0, lambda x, _: x + 1)))

print('{:=^80}'.format(' Example '))

# Iterate over the first record and decode it.
for tfrecord in train_dataset.take(1):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

TRAIN DATASET SIZE: 1000
=================================== Example ====================================
features {
  feature {
    key: "synopsis"
    value {
      bytes_list {
        value: "The Institute of Texan Cultures in San Antonio displays the cultures of the state. Commemorating the scientific genius of Benjamin Franklin, a Philadelphia institute offers interactive exhibits. The DuSable Museum follows African-American history, from World War II\'s Tuskegee airmen to the first black astronaut and mayor. Everyday objects, traditional clothing and the oral history project make up the Ellis Island Immigrant Museum. Human cultures, biology and geology get center stage at the Field Museum of N"
      }
    }
  }
  feature {
    key: "tags"
    value {
      bytes_list {
        value: "Art"
        value: "Educational"
      }
    }
  }
}



In [12]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/tmp/pipelineuss5158a/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [17]:
context.show(statistics_gen.outputs['statistics'])

**NOTE:** Next step, need to create a golden schema. 

In [14]:
infer_schema = SchemaGen(statistics=statistics_gen.outputs['statistics'],
                         infer_feature_shape=True)
context.run(infer_schema)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/tmp/pipelineuss5158a/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

In [15]:
validate_stats = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=infer_schema.outputs['schema']
)
context.run(validate_stats)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/tmp/pipelineuss5158a/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [16]:
context.show(validate_stats.outputs['anomalies'])

**NOTE:** We can get the complete set of tags/labels before we do anything with transform. 

We can either: 
 + Inspect the generated statistics 
 + Inspect the generated schema 
 
The benefit of using the schema is that we will have a "golden" schema that has the entire vocabulary that we know of. Unfortunately, it is quite awkward to get the data from the generated statistics, so for now we'll just worry about the schema. 

In [18]:
num_unique_labels = 151

In [ ]:
!mkdir ../src/schema/

In [ ]:
tfdv.write_schema_text('')

In [19]:
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import dataset_schema

In [20]:
_transform_module_file = 'transform_1.py'

In [21]:
%%writefile {_transform_module_file}
import tensorflow_transform as tft
import tensorflow as tf

FEATURE = 'synopsis'
LABEL = 'tags'

def _transformed_name(key):
    return key + '_xf'

def binarize_tags(transformed_tags, num_labels): 
    """
    Function for turning tags from sparse tensor to multilabel binarized 
    data. The final result is that tags is a binary matrix with shape (none, NUM_TAGS) 
    indicating the presence of a tag in an example
    Args: 
        transformed_tags: sparse tensor with transformed tags 
    Returns: 
        Binarized tags, tensor with shape (none, NUM_TAGS)
    """
    tags_multi_binarized = tf.sparse.to_indicator(transformed_tags, 
                                                  vocab_size=num_labels)
    return tf.cast(tags_multi_binarized, tf.int64)

def preprocessing_fn(inputs, custom_config):
    """Preprocess input columns into transformed columns."""
    outputs = {}
    text = tf.squeeze(inputs[FEATURE], axis=1)
    labels = inputs[LABEL]
    
    num_labels = custom_config.get('num_labels')
    
    # Create and apply a full vocabulary for the labels (subgenres)
    labels = tft.compute_and_apply_vocabulary(
        labels, vocab_filename=LABEL, num_oov_buckets=1
    )

    outputs[FEATURE] = text
    outputs[_transformed_name(LABEL)] = binarize_tags(labels, num_labels)

    return outputs

Overwriting transform_1.py


In [22]:
transform_config = {
    'num_labels': num_unique_labels
}
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=infer_schema.outputs['schema'],
    module_file='./transform_1.py',
    custom_config=transform_config
)
context.run(transform)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
Use ref() instead.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/pipelineuss5158a/Transform/transform_graph/5/.temp_path/tftransform_tmp/00d3e3d5eaae49ba9f0fa7311491141e/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/pipelineuss5158a/Transform/transform_graph/5/.temp_path/tftransform_tmp/16052a33

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:Assets written to: /tmp/pipelineuss5158a/Transform/transform_graph/5/.temp_path/tftransform_tmp/4d567726e1d449f89650eb7ee1760b30/assets
INFO:tensorflow:SavedModel written to: /tmp/pipelineuss5158a/Transform/transform_graph/5/.temp_path/tftransform_tmp/4d567726e1d449f89650eb7ee1760b30/saved_model.pb
value: "\n\013\n\tConst_1:0\022\004tags"

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
value: "\n\013\n\tConst_1:0\022\004tags"

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 13
        uri: "/tmp/pipelineuss5158a/Transform/transform_graph/5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 13
        name: "TransformGraph"
        )]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 5
        uri: "/tmp/pipelineuss5158a/Transform/transformed_examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "/tmp/pipelineuss5158a/Transform/updated_analyzer_cache/5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "TransformCache"
        )]
        ))

In [23]:
transform_dir = transform.outputs['transform_graph'].get()[0].uri
transformed_schema = tfdv.load_schema_text(os.path.join(transform_dir, 'transformed_metadata', 'schema.pbtxt'))

tfdv.display_schema(transformed_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'synopsis',BYTES,required,,-
'tags_xf',INT,required,,-


In [24]:
train_dataset_transformed = make_dataset(transform.outputs['transformed_examples'], 'train')

print('TRAIN DATASET SIZE: {}'.format(train_dataset_transformed.reduce(0, lambda x, _: x + 1)))

tf_transform_output = tft.TFTransformOutput(transform.outputs['transform_graph'].get()[0].uri)

for raw_record in train_dataset_transformed.take(1):
    print('\n {:=^80} \n'.format(" Serialized "))
    print(raw_record)
    print('\n {:=^80} \n'.format(" Decoded "))
    print(tf.io.parse_single_example(raw_record, tf_transform_output.transformed_feature_spec()))

TRAIN DATASET SIZE: 1000

 ================================== Serialized ================================== 

tf.Tensor(b"\n\xc2\x05\n\x93\x04\n\x08synopsis\x12\x86\x04\n\x83\x04\n\x80\x04The Institute of Texan Cultures in San Antonio displays the cultures of the state. Commemorating the scientific genius of Benjamin Franklin, a Philadelphia institute offers interactive exhibits. The DuSable Museum follows African-American history, from World War II's Tuskegee airmen to the first black astronaut and mayor. Everyday objects, traditional clothing and the oral history project make up the Ellis Island Immigrant Museum. Human cultures, biology and geology get center stage at the Field Museum of N\n\xa9\x01\n\x07tags_xf\x12\x9d\x01\x1a\x9a\x01\n\x97\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [25]:
_trainer_module_file = 'model.py'

**NOTE:** We now have no evaluation

In [26]:
%%writefile {_trainer_module_file}
import tensorflow as tf
import tensorflow_transform as tft
import pandas as pd
import numpy as np

from tensorflow.keras import callbacks, layers

from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow_text
import tensorflow_hub as hub
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

# TODO: Add these in config instead of hard-coding
TFHUB_HANDLE_PREPROCESSOR = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
TFHUB_HANDLE_ENCODER = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1"

def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed fies"""
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")

def _input_fn(file_pattern, tf_transform_output, batch_size=64, shuffle=True, epochs=None):
    """Generates features and label for tuning/training.
    Args:
        file_pattern: input tfrecord file pattern.
        tf_transform_output: A TFTransformOutput.
        batch_size: representing the number of consecutive elements of
          returned dataset to combine in a single batch
    Returns:
        A dataset that contains (features, indices) tuple where features
        is a dictionary of Tensors, and indices is a single Tensor of
        label indices.
    """
    transformed_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        shuffle=shuffle,
        label_key='tags_xf',
        num_epochs=epochs
    )
    return dataset

def build_bert_tagger(num_labels):
    # TODO: think about alternative architecture
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='synopsis')
    preprocessing_layer = hub.KerasLayer(TFHUB_HANDLE_PREPROCESSOR, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(TFHUB_HANDLE_ENCODER, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    output = tf.keras.layers.Dense(num_labels, activation="sigmoid")(net)
    return tf.keras.Model(text_input, output)

def get_compiled_model(num_labels):
    # TODO: figure out more about optimizer 
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        model = build_bert_tagger(num_labels)
        metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
        # clipnorm only seems to work in TF 2.4 with distribution strategy 
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.00003,
                                               clipnorm=1,
                                               epsilon=1e-8),
            loss=BinaryCrossentropy(),
            metrics=metrics,
        )
    return model

def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses JSON input"""
    # TODO: Create alternative serving function, especially if using evaluator
    model.tft_layer = tf_transform_output.transform_features_layer()
    tag_vocab= tf_transform_output.vocabulary_by_name('tags')
    
    @tf.function
    def serve_tf_examples_fn(raw_text):
        """Returns the output to be used in the serving signature."""
        reshaped_text = tf.reshape(raw_text, [-1, 1])
        transformed_features = model.tft_layer({"synopsis": reshaped_text})

        outputs = model(transformed_features)
        return {"outputs": outputs[0], "label": tf.constant(tag_vocab)}

    return serve_tf_examples_fn

def run_fn(fn_args):
    """Train the model based on given args
    
    Args:
        fn_args: Holds args used to train the model as name/value pairs
    """
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    # Not sure why its like this
    # TODO: fix this, might be a version issue?
    num_labels = fn_args.custom_config['num_labels']
    
    train_dataset = _input_fn(
        file_pattern=fn_args.train_files,
        tf_transform_output=tf_transform_output,
        batch_size=64)
    
    model = get_compiled_model(num_labels)
    
    # TODO pass in epochs
    history = model.fit(
        train_dataset, 
        epochs=3,
        steps_per_epoch=fn_args.train_steps // 3
    )
    
    signatures = {
        "serving_default": _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")
        ),
    }
    
    model.save(
        fn_args.serving_model_dir, save_format="tf", signatures=signatures
    )

Overwriting model.py


In [27]:
trainer = Trainer(
    module_file=os.path.abspath(_trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=infer_schema.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=3, splits=['train']),
    eval_args=trainer_pb2.EvalArgs(num_steps=0, splits=['train']), 
    custom_config=transform_config) #Need to specify split or it won't work
context.run(trainer)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Epoch 1/3
1/1 [==============================] - ETA: 0s - loss: 0.7442 - precision: 0.0207 - recall: 0.52 - 20s 20s/step - loss: 0.7442 - precision: 0.0207 - recall: 0.5220
Epoch 2/3
1/1 [==============================] - ETA: 0s - loss: 0.7083 - precision: 0.0163 - recall: 0.44 - 9s 9s/step - loss: 0.7083 - precision: 0.0163 - recall: 0.4481
Epoch 3/3
1/1 [==============================] - ETA: 0s - loss: 0.6775 - precision: 0.0199 - recall: 0.46 - 9s 9s/step - loss: 0.6775 - precision: 0.0199 - recall: 0.4611
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Assets written to: /tmp/pipelineuss5158a/Trainer/model/6/serving_model_dir/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 8
        type_id: 16
        uri: "/tmp/pipelineuss5158a/Trainer/model/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "Model"
        )]
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 9
        type_id: 17
        uri: "/tmp/pipelineuss5158a/Trainer/model_run/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "ModelRun"
        )]
        ))

Seems like its quite difficult to do what we want to do in the serving function. Hard to stack things together. 

In [28]:
pusher = Pusher(
  model=trainer.outputs['model'],
  push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving')
  )
)
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 7
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 10
        type_id: 19
        uri: "/tmp/pipelineuss5158a/Pusher/pushed_model/7"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "serving/1615547260"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1615547260"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 19
        name: "PushedModel"
        )]
        ))

In [ ]:
#!gsutil cp -r serving/1614694925/ gs://metadata-bucket-sky/serving-model-dir

Can run the following commands to test the model: 

```bash
    docker run -p 8501:8501 --mount type=bind,source=/home/jupyter/nbcu-metadata-enhancement/training/notebooks/serving/,target=/models/bert-classifier -e MODEL_NAME=bert-classifier -t tensorflow/serving
```

And then in a separate terminal: 

```bash
    curl -d '{"instances": [{"examples": "so much action"}]}' -X POST http://localhost:8501/v1/models/bert-classifier:predict
```

**NOTE:** Check if better to compute vocab in transform and then do transformation in model?

# Generate Some Data For Testing

In [29]:
%%bigquery test_data_df

SELECT 
  synopsis,
  tags
FROM `ml-sandbox-101.metadata_sky.merlin_movie_series_data_small`
LIMIT 100

In [30]:
test_data_df.to_parquet('../src/test_data/test_data.parquet')

In [31]:
pd.read_parquet('../src/test_data/test_data.parquet')

,synopsis,tags
0,The crew heads to Florida for some shiner fish...,"[Sports non-event, Sports, Fishing]"
1,A wealthy adventurer (Cesar Romero) hires the ...,[Crime drama]
2,Grizzy and the Lemmings figure out a way to tr...,"[Comedy, kids (ages 5-9), Action & Adventure, ..."
3,Raisins and sliced almonds enhance spicy Morav...,"[Food, How-To]"
4,Essays and discussions focus on varied views o...,[Public Affairs]
...,...,...
95,A computer-game designer (Ian Ziering) learns ...,[Thriller]
96,Patty and her archenemy compete for a handsome...,"[kids (ages 5-9), Sitcom]"
97,"News and features about performers, production...","[Talk, Entertainment]"
98,"For 6-year-old Luna, the Earth is a giant labo...","[Fantasy, preschoolers (ages 2-4), Science & T..."
